## working with WRF_CHEM:Running WPS

The steps for running WPS as per m2lab tutorial

### Geogrid

- create namelist and geogrid
- copy wps folder and edit namelist for domain 
- use  ncl program to view domain
- ref_lat and ref_lon center point of domain, e_we, e_sn, number of point in x and y direction
- edit geog file location in name list and geogrid outptut location
- then run geogrid.exe, run geogrid after editing the name list as ./geogrid.exe, in wps folder

### ungrib

- UNGRIB-STEP1 ./link_grib.csh /home/swl-sacon-dst/wrf/WRF341/TEST/fnl/ #to make link for ungrib ls -alth GRIB* #to view how the links are made, it seems it make links for all the fnl files available in the #folder util/g2print.exe GRIBFILE.AAA #to view the variable in the linked files
- UNGRIB-STEP2 #have to make vtables, no vtable ending with gfs was foundi n the folder cd /WRF/MYTEST/WPS/ungrib/vtables #but latter found that form [here](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/DATA/FNL/) gfs vtable is same #for this also, so made cp ungrib/Variable_Tables/Vtable.GFS Vtable #then executed ./ungrib.exe #run for 30 seconds, then given the good nes of succefful completion of ungirb #then run this util/rd_intermediate.exe FILE:2008?04?25_00 #succefully showed the detials of the file

### Metgrid

- running metgrid gives error of At line 98 of file gridinfo_module.f90 (unit = 10, file = 'namelist.wps')
Fortran runtime error: Cannot match namelist object name gfs
#changed namelist.wps as per the m2lab and now the error becomes RROR: Could not open file METGRID.TBL application called MPI_Abort(MPI_COMM_WORLD, 0) - process 0
- so thought that it wouldb ebecause of mpi copilation of wrf-chem and so run mpirun -np 1 ./metgrid.exe
it gives `mpirun: error while loading shared libraries: libopen-rte.so.4: cannot open shared object file: No such file or directory`, googled and found [this](http://juniarto.blogspot.in/2012/08/mpirun-error-while-loading-shared.html) as per this run `sudo ldconfig` and now runnin gthis command gives `RROR: Could not open file METGRID.TBL application called MPI_Abort(MPI_COMM_WORLD, 0) - process 0`
- it seems error is from two lines in namelist.wps 
```bash
opt_output_from_metgrid_path = '/home/swl-sacon-dst/wrf/WRF341/TEST/WPS/wpsprd'
opt_metgrid_tbl_path       = '/home/swl-sacon-dst/wrf/WRF341/TEST/WPS/wpsprd'
 ``` 
saying location of metgrid.TBL file, so removed that and run, showing some error in fortran file ```gridinfo_module.f90```
- so in review it is found that th erog data file path is worng and there is no erod file in it. it is reflected in geogrid/log also no erod was processed as told in wrf chem tutorial
- so added the eord folder in namelist.wps and linked the data, also made the METGRID.TBL link from CHEM.TBL as per the tutorial1 of wrf chem and rerun the ungrib.exe but the log doesn't showing any changes. later it is found that the geogrid.exe also has to run again so deleted the geogrid.exe optuput file and rerun geogrid.exe, this time it gives error of 
```bash
./geogrid.exe
Parsed 22 entries in GEOGRID.TBL
Processing domain 1 of 4
ERROR: Could not open /home/swl-sacon-dst/wrf/WRF341/TEST/GEOG/topo_10m/index
application called MPI_Abort(MPI_COMM_WORLD, 0) - process 0
```
- from wrf ems it is found that the geog folder is the largest file folder in wrf ems for making high resoltion geoplots. so made a link to this folder in namelist and rerun the geogrid and now erod was processed.  then run link script, ungrib and metgrid.exe. In runnning metgrid it exited with EOF, later it found that it is due to a typo'missing ' ' in the wps file. Now wps is running without any errors.
- the metfile are made in the wps folder itself